# Segmentierung mit Stardist
Dieses Notebook ist dem 2D-Beispiel https://github.com/mpicbg-csbd/stardist/tree/master/examples/2D der GitHub Implementierung entnommen.

Nun wenden wir uns der Segmentierung mit DeepLearning zu. Hier verwenden wir wieder die *E. Coli* Daten.

In [ ]:
from __future__ import print_function, unicode_literals, absolute_import, division
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file

from stardist import fill_label_holes, relabel_image_stardist, random_label_cmap
from stardist.matching import matching_dataset

#np.random.seed(42)
lbl_cmap = random_label_cmap()

# Data

This notebook demonstrates how the training data for *StarDist* should look like and whether the annotated objects can be appropriately described by star-convex polygons. 

<div class="alert alert-block alert-info">
The training data that needs to be provided for StarDist consists of corresponding pairs of raw images and pixelwise annotated ground truth images (masks), where every pixel has a unique integer value indicating the object id (or 0 for background). 
</div>

In [ ]:
X_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/input/*.tif'))
Y_glob = sorted(glob('/extdata/readonly/f-prak-v15/e-coli-swarming/train/labels/*.tif'))
def labelname(name):
    return name[:-5]+name[-4:]
assert all(Path(x).name == labelname(Path(y).name) for x,y in zip(X_glob, Y_glob))

Load only a small subset

In [ ]:
X_glob, Y_glob = X_glob[:10], Y_glob[:10]

In [ ]:
X = list(map(imread, X_glob))
Y = list(map(imread, Y_glob))

# Example image

In [ ]:
i = min(4, len(X)-1)
img, lbl = X[i], fill_label_holes(Y[i])
assert img.ndim in (2,3)
img = img if img.ndim==2 else img[...,:3]
# assumed axes ordering of img and lbl is: YX(C)

In [ ]:
plt.figure(figsize=(16,10))
plt.subplot(121); plt.imshow(img,cmap='gray');   plt.axis('off'); plt.title('Raw image')
plt.subplot(122); plt.imshow(lbl,cmap=lbl_cmap); plt.axis('off'); plt.title('GT labels')
None;

# Fitting ground-truth labels with star-convex polygons
In unseren Ground Truth labels sind Objekte einfach zusammenhängende Pixel. Stardist betrachtet Objekte jedoch als Polygone. Macht euch diesen Unterschied nochmals klar! Im Folgenden ist `n_rays` die Anzahl der Strahlen, also auch die Anzahl der Polygonpunkte, mit der ein Objekt dargstellt wird. Es wird überprüft, wie viele Polygonpunkte benötigt werden, um die Objekte auch mit der Polygon Darstellung gut zu beschreiben. Dafür wird die Interscetion over Union (IoU) von label-Objekt mit dem Polygon-Objekt berechnet für verschiedene Anzahlen an Strahlen.

In [ ]:
n_rays = [2**i for i in range(2,8)]
scores = []
for r in tqdm(n_rays):
    Y_reconstructed = [relabel_image_stardist(lbl, n_rays=r) for lbl in Y]
    mean_iou = matching_dataset(Y, Y_reconstructed, thresh=0, show_progress=False).mean_true_score
    scores.append(mean_iou)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(n_rays, scores, 'o-')
plt.xlabel('Number of rays for star-convex polygon')
plt.ylabel('Reconstruction score (mean intersection over union)')
plt.title("Accuracy of ground truth reconstruction (should be > 0.8 for a reasonable number of rays)")
None;

## Example image reconstructed with various number of rays

In [ ]:
fig, ax = plt.subplots(2,3, figsize=(16,11))
for a,r in zip(ax.flat,n_rays):
    a.imshow(relabel_image_stardist(lbl, n_rays=r), cmap=lbl_cmap)
    a.set_title('Reconstructed (%d rays)' % r)
    a.axis('off')
plt.tight_layout();